In [4]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Paso 1: Leer todas las imágenes y mezclarlas
def cargar_y_mezclar_imagenes(directorio):
    imagenes = []
    tamaño = (128, 128)  # Definir tamaño de la imagen como 128x128
    for clase in os.listdir(directorio):
        for filename in os.listdir(os.path.join(directorio, clase)):
            img = cv2.imread(os.path.join(directorio, clase, filename))
            if img is not None:
                img = cv2.resize(img, tamaño)  # Redimensionar a un tamaño uniforme
                imagenes.append(img)
    np.random.shuffle(imagenes)
    return imagenes

# Paso 2: Convertir imágenes a un formato adecuado (aplanar las imágenes)
def convertir_a_vectores(imagenes):
    return np.array([img.flatten() for img in imagenes])

# Paso 3: Aplicar Kernel PCA
def aplicar_kpca(datos, n_componentes, kernel='rbf', gamma=0.01):
    kpca = KernelPCA(n_components=n_componentes, kernel=kernel, gamma=gamma)
    datos_kpca = kpca.fit_transform(datos)
    return datos_kpca, kpca

# Paso 4: Reducción de características con PCA
def aplicar_pca(datos, n_componentes):
    pca = PCA(n_components=n_componentes)
    datos_pca = pca.fit_transform(datos)
    return datos_pca, pca

# Paso 5: Entrenamiento de modelos KMeans y DBSCAN
def entrenar_modelos(datos, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(datos)
    
    dbscan = DBSCAN(eps=0.5, min_samples=5)  # Ajustar estos parámetros según sea necesario
    dbscan.fit(datos)
    
    return kmeans, dbscan

# Paso 6: Evaluación de la calidad de la agrupación
def evaluar_agrupacion(modelo, datos):
    n_clusters = len(set(modelo.labels_)) - (1 if -1 in modelo.labels_ else 0)
    if n_clusters < 2:
        return None, None  # No es posible calcular los puntajes con menos de 2 clusters
    silhouette = silhouette_score(datos, modelo.labels_)
    calinski_harabasz = calinski_harabasz_score(datos, modelo.labels_)
    return silhouette, calinski_harabasz

# Directorio del dataset
directorio = "dataset"

# Leer y mezclar imágenes
dataset = cargar_y_mezclar_imagenes(directorio)

# Convertir imágenes a vectores
X = convertir_a_vectores(dataset)

# Aplicar Kernel PCA
n_componentes_kpca = 100
X_kpca, kpca = aplicar_kpca(X, n_componentes_kpca)

# Aplicar PCA (alternativamente puedes comentar esta línea si solo quieres usar KPCA)
n_componentes_pca = 100
X_pca, pca = aplicar_pca(X, n_componentes_pca)

# Entrenar modelos de clustering con KPCA
n_clusters = 3
kmeans_kpca_model, dbscan_kpca_model = entrenar_modelos(X_kpca, n_clusters)

# Evaluar modelos de clustering con KPCA
silhouette_kmeans_kpca, calinski_kmeans_kpca = evaluar_agrupacion(kmeans_kpca_model, X_kpca)
silhouette_dbscan_kpca, calinski_dbscan_kpca = evaluar_agrupacion(dbscan_kpca_model, X_kpca)

# Entrenar modelos de clustering con PCA
kmeans_pca_model, dbscan_pca_model = entrenar_modelos(X_pca, n_clusters)

# Evaluar modelos de clustering con PCA
silhouette_kmeans_pca, calinski_kmeans_pca = evaluar_agrupacion(kmeans_pca_model, X_pca)
silhouette_dbscan_pca, calinski_dbscan_pca = evaluar_agrupacion(dbscan_pca_model, X_pca)

# Imprimir resultados
print("Resultados del modelo KMeans con KPCA:")
print("Silhouette Score:", silhouette_kmeans_kpca)
print("Calinski-Harabasz Score:", calinski_kmeans_kpca)

print("\nResultados del modelo DBSCAN con KPCA:")
if silhouette_dbscan_kpca is not None:
    print("Silhouette Score:", silhouette_dbscan_kpca)
    print("Calinski-Harabasz Score:", calinski_dbscan_kpca)
else:
    print("DBSCAN con KPCA no encontró suficientes clusters para evaluación")

print("\nResultados del modelo KMeans con PCA:")
print("Silhouette Score:", silhouette_kmeans_pca)
print("Calinski-Harabasz Score:", calinski_kmeans_pca)

print("\nResultados del modelo DBSCAN con PCA:")
if silhouette_dbscan_pca is not None:
    print("Silhouette Score:", silhouette_dbscan_pca)
    print("Calinski-Harabasz Score:", calinski_dbscan_pca)
else:
    print("DBSCAN con PCA no encontró suficientes clusters para evaluación")


Resultados del modelo KMeans con KPCA:
Silhouette Score: 0.6757006197834727
Calinski-Harabasz Score: 30.257316307310663

Resultados del modelo DBSCAN con KPCA:
DBSCAN con KPCA no encontró suficientes clusters para evaluación

Resultados del modelo KMeans con PCA:
Silhouette Score: 0.15706827164485607
Calinski-Harabasz Score: 310.358795532333

Resultados del modelo DBSCAN con PCA:
DBSCAN con PCA no encontró suficientes clusters para evaluación


C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Usuario\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
